In [2]:
import scipy.io as sio
from scipy.misc import imread
from preprocess.normalize import preprocess_signature
import tf_signet
from tf_cnn_model import TF_CNNModel
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import os
import scipy.io
from find_largest_image import find_largest
import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
import random
from numpy.random import choice
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import sklearn.pipeline as pipeline
import sklearn.preprocessing as preprocessing

In [3]:
data_folder = 'C:\\Users\\Mert\\Documents\\GitHub\\sigver_bmg\\data\\downloaded_pp_features\\gpds_signet_all'

In [4]:
data_f = pd.read_csv(os.path.join(data_folder,'data_features.csv'))
visual_f = pd.read_csv(os.path.join(data_folder,'visual_features.csv'))

# MODEL SELECTION & TRAINING

In [7]:
fakes_preds = []
gens_preds = []

for fold in np.arange(0,10):
    user_numbers = data_f['user_id'].unique()
    np.random.shuffle(user_numbers)
    dev_user_ids = user_numbers[0:531]
    validation_user_ids = user_numbers[531:581]
    exploitation_user_ids = user_numbers[581:]

    dev_df = data_f.loc[data_f['user_id'].isin(dev_user_ids)]
    dev_vf = visual_f.loc[dev_df.index]
    val_df = data_f.loc[data_f['user_id'].isin(validation_user_ids)]
    val_vf = visual_f.loc[val_df.index]
    exp_df = data_f.loc[data_f['user_id'].isin(exploitation_user_ids)]
    exp_vf = visual_f.loc[exp_df.index]

    dev_df_gen = dev_df.loc[dev_df['fakeness']==0]
    dev_df_fake = dev_df.loc[dev_df['fakeness']==1]
    dev_df_gen_12 = dev_df_gen.loc[dev_df_gen['sig_id'].isin(np.arange(1,13))]
    dev_df_valid_12 = dev_df_gen.loc[dev_df_gen['sig_id'].isin(np.arange(13,25))]

    val_df_gen = val_df.loc[val_df['fakeness']==0]
    val_df_fake = val_df.loc[val_df['fakeness']==1]
    val_df_gen_12 = val_df_gen.loc[val_df_gen['sig_id'].isin(np.arange(1,13))]
    val_df_valid_gen_12 = val_df_gen.loc[val_df_gen['sig_id'].isin(np.arange(13,25))]


    for user_id in tqdm.tqdm(validation_user_ids, ascii=True):
        orig_clf = MLPClassifier(hidden_layer_sizes=(100,50))
        clf = pipeline.Pipeline([('scaler', preprocessing.StandardScaler(with_mean=False)), ('classifier', orig_clf)])
        y_train = (pd.concat([val_df_gen_12.loc[val_df_gen_12['user_id']==user_id],dev_df_gen]))['user_id']==user_id
        X_train = visual_f.loc[y_train.index]  
        clf.fit(X_train, y_train)
        y_valid_fakes = val_df_fake.loc[(val_df_fake['user_id']==user_id)]
        X_valid_f = visual_f.loc[y_valid_fakes.index]
        fakes_preds.append(clf.predict_proba(X_valid_f))
        y_valid_gens = val_df_valid_gen_12.loc[val_df_valid_gen_12['user_id']==user_id]
        X_valid_g = visual_f.loc[y_valid_gens.index]
        gens_preds.append(clf.predict_proba(X_valid_g))




  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

  2%|#6                                                                                 | 1/50 [00:12<10:32, 12.92s/it]

  4%|###3                                                                               | 2/50 [00:26<10:35, 13.24s/it]

  6%|####9                                                                              | 3/50 [00:41<10:37, 13.56s/it]

  8%|######6                                                                            | 4/50 [00:55<10:31, 13.72s/it]

 10%|########3                                                                          | 5/50 [01:07<09:54, 13.22s/it]

 12%|#########9                                                                         | 6/50 [01:21<09:57, 13.59s/it]

 14%|###########6                                                                       | 7/50 [01:35<09:45, 13.62s/it]

 16%|#############2           

 32%|##########################2                                                       | 16/50 [03:27<07:03, 12.44s/it]

 34%|###########################8                                                      | 17/50 [03:40<06:51, 12.47s/it]

 36%|#############################5                                                    | 18/50 [03:52<06:33, 12.29s/it]

 38%|###############################1                                                  | 19/50 [04:04<06:19, 12.25s/it]

 40%|################################8                                                 | 20/50 [04:16<06:05, 12.19s/it]

 42%|##################################4                                               | 21/50 [04:28<05:50, 12.10s/it]

 44%|####################################                                              | 22/50 [04:40<05:41, 12.21s/it]

 46%|#####################################7                                            | 23/50 [04:55<05:49, 12.96s/it]

 48%|###########################

 64%|####################################################4                             | 32/50 [06:19<03:28, 11.60s/it]

 66%|######################################################1                           | 33/50 [06:31<03:18, 11.69s/it]

 68%|#######################################################7                          | 34/50 [06:42<03:05, 11.58s/it]

 70%|#########################################################4                        | 35/50 [06:54<02:56, 11.73s/it]

 72%|###########################################################                       | 36/50 [07:06<02:46, 11.86s/it]

 74%|############################################################6                     | 37/50 [07:18<02:32, 11.76s/it]

 76%|##############################################################3                   | 38/50 [07:29<02:21, 11.76s/it]

 78%|###############################################################9                  | 39/50 [07:41<02:08, 11.66s/it]

 80%|###########################

 96%|##############################################################################7   | 48/50 [08:52<00:21, 10.86s/it]

 98%|################################################################################3 | 49/50 [09:02<00:10, 10.62s/it]

100%|##################################################################################| 50/50 [09:12<00:00, 10.55s/it]

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

  2%|#6                                                                                 | 1/50 [00:11<09:24, 11.51s/it]

  4%|###3                                                                               | 2/50 [00:22<08:58, 11.22s/it]

  6%|####9                                                                              | 3/50 [00:32<08:29, 10.84s/it]

  8%|######6                                                                            | 4/50 [00:42<08:13, 10.72s/it]

 10%|########3                  

 26%|#####################3                                                            | 13/50 [02:34<07:08, 11.59s/it]

 28%|######################9                                                           | 14/50 [02:45<06:56, 11.58s/it]

 30%|########################5                                                         | 15/50 [02:58<07:00, 12.01s/it]

 32%|##########################2                                                       | 16/50 [03:10<06:47, 11.98s/it]

 34%|###########################8                                                      | 17/50 [03:21<06:26, 11.72s/it]

 36%|#############################5                                                    | 18/50 [03:32<06:08, 11.51s/it]

 38%|###############################1                                                  | 19/50 [03:44<05:56, 11.51s/it]

 40%|################################8                                                 | 20/50 [03:56<05:50, 11.69s/it]

 42%|###########################

 58%|###############################################5                                  | 29/50 [05:29<03:44, 10.67s/it]

 60%|#################################################1                                | 30/50 [05:41<03:43, 11.17s/it]

 62%|##################################################8                               | 31/50 [05:53<03:37, 11.45s/it]

 64%|####################################################4                             | 32/50 [06:06<03:34, 11.92s/it]

 66%|######################################################1                           | 33/50 [06:20<03:32, 12.48s/it]

 68%|#######################################################7                          | 34/50 [06:33<03:22, 12.65s/it]

 70%|#########################################################4                        | 35/50 [06:45<03:08, 12.55s/it]

 72%|###########################################################                       | 36/50 [06:58<02:54, 12.48s/it]

 74%|###########################

 90%|#########################################################################8        | 45/50 [07:41<00:49,  9.97s/it]

 92%|###########################################################################4      | 46/50 [07:51<00:39,  9.96s/it]

 94%|#############################################################################     | 47/50 [08:01<00:29,  9.90s/it]

 96%|##############################################################################7   | 48/50 [08:11<00:20, 10.05s/it]

 98%|################################################################################3 | 49/50 [08:21<00:09,  9.95s/it]

100%|##################################################################################| 50/50 [08:32<00:00, 10.10s/it]

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

  2%|#6                                                                                 | 1/50 [00:09<08:05,  9.91s/it]

  4%|###3                       

 20%|################4                                                                 | 10/50 [01:50<07:36, 11.41s/it]

 22%|##################                                                                | 11/50 [02:00<07:07, 10.97s/it]

 24%|###################6                                                              | 12/50 [02:11<06:52, 10.84s/it]

 26%|#####################3                                                            | 13/50 [02:21<06:38, 10.78s/it]

 28%|######################9                                                           | 14/50 [02:31<06:18, 10.52s/it]

 30%|########################5                                                         | 15/50 [02:41<06:01, 10.33s/it]

 32%|##########################2                                                       | 16/50 [02:51<05:46, 10.18s/it]

 34%|###########################8                                                      | 17/50 [03:01<05:37, 10.24s/it]

 36%|###########################

# THRESHOLD SELECTION

In [11]:
DECISION_THRESHOLD = 0.5
tn = 0
fp = 0
for val in fakes_preds:
    for pred in val:
        if pred[1] <= DECISION_THRESHOLD: tn += 1
        else : fp += 1

print("fakes error rate:",(1-(float(tn)/(tn+fp)))*100)
        
tp = 0
fn = 0
for val in gens_preds:
    for pred in val:
        if pred[1] >= DECISION_THRESHOLD: tp += 1
        else : fn += 1

print("gens error rate:",(1-(float(tp)/(tp+fn)))*100)

fakes error rate: 8.828454685511423
gens error rate: 10.666666666666668


In [9]:
pred

array([9.99956718e-01, 4.32818109e-05])